In [145]:
import pandas as pd
import re

In [146]:
df_raw = pd.read_csv('data/kor/attraction_review.csv',
                     names=['placeId','userId','score','title','review'],
                     encoding='ms949')
df_raw = df_raw.drop(df_raw.columns[5:],axis=1)
df_raw = df_raw.dropna()
df_raw.score = df_raw.score/10
df_raw.score = df_raw.score.apply(int)
df_raw.review = df_raw.title + ' ' + df_raw.review
df_raw = df_raw.drop(columns=['title'])
df_raw.review = df_raw.review.apply(lambda x: re.sub('(<br/>|[ ]+)',' ',x))
df_raw = df_raw.reset_index(drop=True)
df_raw.to_csv('data/kor/attraction_review_pp.csv',mode='w')

In [147]:
df_raw = pd.read_csv('data/kor/hotel_review.csv',
                     names=['placeId','userId','score','title','review'],
                     encoding='ms949')
df_raw = df_raw.drop(df_raw.columns[5:],axis=1)
df_raw = df_raw.dropna()
df_raw.score = df_raw.score/10
df_raw.score = df_raw.score.apply(int)
df_raw.review = df_raw.title + ' ' + df_raw.review
df_raw = df_raw.drop(columns=['title'])
df_raw.review = df_raw.review.apply(lambda x: re.sub('(<br/>|[ ]+)',' ',x))
df_raw = df_raw.reset_index(drop=True)
df_raw.to_csv('data/kor/hotel_review_pp.csv',mode='w')

In [148]:
df_raw = pd.read_csv('data/kor/restaurant_review.csv',
                     names=['placeId','userId','score','title','review'],
                    encoding='ms949')
df_raw = df_raw.drop(df_raw.columns[5:],axis=1)
df_raw = df_raw.dropna()
df_raw.score = df_raw.score/10
df_raw.score = df_raw.score.apply(int)
df_raw.review = df_raw.title + ' ' + df_raw.review
df_raw = df_raw.drop(columns=['title'])
df_raw.review = df_raw.review.apply(lambda x: re.sub('(<br/>|[ ]+)',' ',x))
#유니코드 replace
def unicode(match):
    match = match.group()
    return chr(int(match[2:], base=16))
for i in df_raw.userId:
    i = re.sub(r'\\u[0-9a-f]{4}', unicode, i)
df_raw.userId = df_raw.userId.apply(lambda x:
                    re.sub(r'\\u[0-9a-f]{4}',
                           unicode, x))
df_raw = df_raw.reset_index(drop=True)
df_raw.to_csv('data/kor/restaurant_review_pp.csv',mode='w')